# Keras MobileNet Benchmark

In a previous benchmark we used a simple three layer ConvNet. This time we use a deeper MobileNet architecture on greyscale strokes. 

This kernel has three main components:

* MobileNet
* Fast and memory efficient Image Generator with temporal 
* Full training & submission with Kaggle Kernel

I did some paramer search but it should not be hard to improve the current score.

## Setup
Import the necessary libraries and a few helper functions.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import json
import os
import ast
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam, AdamAccumulate
from keras.applications import MobileNet, InceptionResNetV2, InceptionV3, Xception, NASNetMobile, DenseNet121, DenseNet201, ResNet50
from keras.applications.mobilenet import preprocess_input
from keras.applications.mobilenetv2 import MobileNetV2
from keras import backend as K
start = dt.datetime.now()

Using TensorFlow backend.


In [2]:
DP_DIR = 'input/shuffle_csvs/'
INPUT_DIR = 'input'

BASE_SIZE = 256
NCSVS = 100
NCATS = 340
#np.random.seed(seed=1987)
#tf.set_random_seed(seed=1987)
def f2cat(filename: str) -> str:
    return filename.split('.')[0]

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, 'train_simplified'))
    return sorted([f2cat(f) for f in files], key=str.lower)

In [3]:
def apk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

## MobileNet

MobileNets are based on a streamlined architecture that uses depthwise separable convolutions to build light weight deep neural networks.

[MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://arxiv.org/pdf/1704.04861.pdf)

In [4]:
STEPS = 10000
EPOCHS = 1
size = 112
batchsize = 128

In [6]:
base_model = ResNet(input_shape=(size, size, 3), weights='imagenet', include_top=False)
y = base_model.output
y = GlobalAveragePooling2D(name='avg_pool')(y)
y = Dense(NCATS, activation='softmax')(y)
model = Model(inputs=base_model.input, outputs=y)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 55, 55, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 55, 55, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 55, 55, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [7]:
model.load_weights('ftw/good_xception.h5')

In [8]:
for layer in model.layers[:-40]:
   layer.trainable = False
            
batchsize=340
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 192, 192, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 95, 95, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 95, 95, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 95, 95, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [9]:
#model.compile(optimizer=AdamAccumulate(lr=0.002, accum_iters=4), loss='categorical_crossentropy',
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',
            metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])

## Training with Image Generator

In [7]:
def draw_cv2(raw_strokes, size=256, lw=6, time_color=True):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    rgb = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    if size != BASE_SIZE:
        return cv2.resize(rgb, (size, size))
    else:
        return rgb

def image_generator_xd(size, batchsize, ks, lw=6, time_color=True):
    while True:
        for k in np.random.permutation(ks):
            filename = os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(k))
            for df in pd.read_csv(filename, chunksize=batchsize):
                df['drawing'] = df['drawing'].apply(json.loads)
                x = np.zeros((len(df), size, size, 3))
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, :3] = draw_cv2(raw_strokes, size=size, lw=lw,
                                             time_color=time_color)
                x = preprocess_input(x).astype(np.float32)
                y = keras.utils.to_categorical(df.y, num_classes=NCATS)
                yield x, y

def df_to_image_array_xd(df, size, lw=6, time_color=True):
    df['drawing'] = df['drawing'].apply(json.loads)
    x = np.zeros((len(df), size, size, 3))
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, :3] = draw_cv2(raw_strokes, size=size, lw=lw, time_color=time_color)
    x = preprocess_input(x).astype(np.float32)
    return x

def image_generator_test(size, batchsize, lw=6, time_color=True):
    while True:
            filename = 'input/test_simplified.csv'
            for df in pd.read_csv(filename, chunksize=batchsize):
                df['drawing'] = df['drawing'].apply(json.loads)
                x = np.zeros((len(df), size, size, 3))
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, :3] = draw_cv2(raw_strokes, size=size, lw=lw,
                                             time_color=time_color)
                x = preprocess_input(x).astype(np.float32)
                yield x
                
def draw_cv2_1(raw_strokes, size=256, lw=6, time_color=True):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

def image_generator_xd_1(size, batchsize, ks, lw=6, time_color=True):
    while True:
        for k in np.random.permutation(ks):
            filename = os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(k))
            cnt = 0
            for df in pd.read_csv(filename, chunksize=batchsize):
                cnt = cnt + batchsize
                if cnt >= 34000:
                    break
                df['drawing'] = df['drawing'].apply(json.loads)
                x = np.zeros((len(df), size, size, 3))
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, :3] = draw_cv2(raw_strokes, size=size, lw=lw,
                                             time_color=time_color)
                x = preprocess_input(x).astype(np.float32)
                y = keras.utils.to_categorical(df.y, num_classes=NCATS)
                yield x, y



def df_to_image_array_xd_1(df, size, lw=6, time_color=True):
    df['drawing'] = df['drawing'].apply(json.loads)
    x = np.zeros((len(df), size, size, 1))
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw, time_color=time_color)
    x = preprocess_input(x).astype(np.float32)
    return x

def image_generator_test_1(size, batchsize, lw=6, time_color=True):
    while True:
            filename = 'input/test_simplified.csv'
            for df in pd.read_csv(filename, chunksize=batchsize):
                df['drawing'] = df['drawing'].apply(json.loads)
                x = np.zeros((len(df), size, size, 1))
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw,
                                             time_color=time_color)
                x = preprocess_input(x).astype(np.float32)
                yield x

In [8]:
valid_df = pd.read_csv(os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(NCSVS - 1)), nrows=34000)
valid_datagen = image_generator_xd_1(size=size, batchsize=batchsize, ks=[NCSVS-1])

In [9]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, ks=range(NCSVS - 1))

In [13]:
#model = load_model('ftw/good192.h5', custom_objects={'top_3_accuracy': top_3_accuracy, 'AdamAccumulate': AdamAccumulate})
#model.summary()
#model.save_weights('ftw/good192_weights.h5')
#model.load_weights('ftw/good192.h5', by_name=True)
#model = load_model('weights_checkpoints_resnet50/weights_resnet50_full_data_256_FINAL_2nd_try.04-0.6527.h5', custom_objects={'top_3_accuracy': top_3_accuracy, 'AdamAccumulate': AdamAccumulate})

In [ ]:
checkpointer = keras.callbacks.ModelCheckpoint(filepath='xception/xception.{epoch:02d}-{val_loss:.4f}.h5', save_weights_only=False, verbose=1)
callbacks = [
#    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
#                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    checkpointer
]
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=1000, epochs=50, verbose=1,
    validation_data = valid_datagen,
    validation_steps = 34000 / batchsize,
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/50
1000/1000 [==============================] - 1676s 2s/step - loss: 1.1646 - categorical_crossentropy: 1.1646 - categorical_accuracy: 0.7030 - top_3_accuracy: 0.8689 - val_loss: 1.1379 - val_categorical_crossentropy: 1.1379 - val_categorical_accuracy: 0.7051 - val_top_3_accuracy: 0.8752

Epoch 00001: saving model to xception/xception.01-1.1379.h5
Epoch 2/50
1000/1000 [==============================] - 1670s 2s/step - loss: 0.8564 - categorical_crossentropy: 0.8564 - categorical_accuracy: 0.7765 - top_3_accuracy: 0.9156 - val_loss: 1.0273 - val_categorical_crossentropy: 1.0273 - val_categorical_accuracy: 0.7304 - val_top_3_accuracy: 0.8935

Epoch 00002: saving model to xception/xception.02-1.0273.h5
Epoch 3/50
 634/1000 [==================>...........] - ETA: 9:28 - loss: 0.8137 - categorical_crossentropy: 0.8137 - categorical_accuracy: 0.7877 - top_3_accuracy: 0.9206

In [14]:
from keras import backend as K

K.set_value(model.optimizer.lr, 0.00025)

hist = model.fit_generator(
    train_datagen, steps_per_epoch=1000, epochs=20, verbose=1,
    validation_data = valid_datagen,
    validation_steps = 34000 / batchsize,
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/20
1000/1000 [==============================] - 774s 774ms/step - loss: 0.6690 - categorical_crossentropy: 0.6690 - categorical_accuracy: 0.8248 - top_3_accuracy: 0.9379 - val_loss: 0.6690 - val_categorical_crossentropy: 0.6690 - val_categorical_accuracy: 0.8248 - val_top_3_accuracy: 0.9388

Epoch 00001: saving model to xception/xception.01-0.6690.h5
Epoch 2/20
1000/1000 [==============================] - 775s 775ms/step - loss: 0.6946 - categorical_crossentropy: 0.6946 - categorical_accuracy: 0.8193 - top_3_accuracy: 0.9365 - val_loss: 0.6677 - val_categorical_crossentropy: 0.6677 - val_categorical_accuracy: 0.8242 - val_top_3_accuracy: 0.9388

Epoch 00002: saving model to xception/xception.02-0.6677.h5
Epoch 3/20
1000/1000 [==============================] - 776s 776ms/step - loss: 0.6806 - categorical_crossentropy: 0.6806 - categorical_accuracy: 0.8236 - top_3_accuracy: 0.9380 - val_loss: 0.6673 - val_categorical_crossentropy: 0.6673 - val_categorical_accuracy: 0.8240 - val_t

KeyboardInterrupt: 

In [15]:
K.set_value(model.optimizer.lr, 0.000125)

hist = model.fit_generator(
    train_datagen, steps_per_epoch=1000, epochs=10, verbose=1,
    validation_data = valid_datagen,
    validation_steps = 34000 / batchsize,
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/10
1000/1000 [==============================] - 777s 777ms/step - loss: 0.6771 - categorical_crossentropy: 0.6771 - categorical_accuracy: 0.8217 - top_3_accuracy: 0.9366 - val_loss: 0.6624 - val_categorical_crossentropy: 0.6624 - val_categorical_accuracy: 0.8250 - val_top_3_accuracy: 0.9388

Epoch 00001: saving model to xception/xception.01-0.6624.h5
Epoch 2/10
1000/1000 [==============================] - 777s 777ms/step - loss: 0.6603 - categorical_crossentropy: 0.6603 - categorical_accuracy: 0.8253 - top_3_accuracy: 0.9392 - val_loss: 0.6632 - val_categorical_crossentropy: 0.6632 - val_categorical_accuracy: 0.8248 - val_top_3_accuracy: 0.9389

Epoch 00002: saving model to xception/xception.02-0.6632.h5
Epoch 3/10
 455/1000 [============>.................] - ETA: 6:02 - loss: 0.6728 - categorical_crossentropy: 0.6728 - categorical_accuracy: 0.8232 - top_3_accuracy: 0.9380

KeyboardInterrupt: 

In [16]:
K.set_value(model.optimizer.lr, 0.0000625)

hist = model.fit_generator(
    train_datagen, steps_per_epoch=1000, epochs=10, verbose=1,
    validation_data = valid_datagen,
    validation_steps = 34000 / batchsize,
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/10
1000/1000 [==============================] - 736s 736ms/step - loss: 0.6459 - categorical_crossentropy: 0.6459 - categorical_accuracy: 0.8314 - top_3_accuracy: 0.9403 - val_loss: 0.6335 - val_categorical_crossentropy: 0.6335 - val_categorical_accuracy: 0.8309 - val_top_3_accuracy: 0.9432

Epoch 00001: saving model to xception/xception.01-0.6335.h5
Epoch 2/10
1000/1000 [==============================] - 735s 735ms/step - loss: 0.6402 - categorical_crossentropy: 0.6402 - categorical_accuracy: 0.8330 - top_3_accuracy: 0.9410 - val_loss: 0.6341 - val_categorical_crossentropy: 0.6341 - val_categorical_accuracy: 0.8314 - val_top_3_accuracy: 0.9429

Epoch 00002: saving model to xception/xception.02-0.6341.h5
Epoch 3/10
1000/1000 [==============================] - 735s 735ms/step - loss: 0.6299 - categorical_crossentropy: 0.6299 - categorical_accuracy: 0.8319 - top_3_accuracy: 0.9421 - val_loss: 0.6329 - val_categorical_crossentropy: 0.6329 - val_categorical_accuracy: 0.8319 - val_t

In [22]:
K.set_value(model.optimizer.lr, 0.00025)

hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=10, verbose=1,
    validation_data = valid_datagen,
    validation_steps = 34000 / batchsize,
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/10
10000/10000 [==============================] - 5850s 585ms/step - loss: 0.6667 - categorical_crossentropy: 0.6667 - categorical_accuracy: 0.8254 - top_3_accuracy: 0.9382 - val_loss: 0.6734 - val_categorical_crossentropy: 0.6734 - val_categorical_accuracy: 0.8241 - val_top_3_accuracy: 0.9379

Epoch 00001: saving model to xception/xception.01-0.6734.h5
Epoch 2/10
10000/10000 [==============================] - 5854s 585ms/step - loss: 0.6624 - categorical_crossentropy: 0.6624 - categorical_accuracy: 0.8263 - top_3_accuracy: 0.9384 - val_loss: 0.6544 - val_categorical_crossentropy: 0.6544 - val_categorical_accuracy: 0.8286 - val_top_3_accuracy: 0.9389

Epoch 00002: saving model to xception/xception.02-0.6544.h5
Epoch 3/10
10000/10000 [==============================] - 5854s 585ms/step - loss: 0.6612 - categorical_crossentropy: 0.6612 - categorical_accuracy: 0.8269 - top_3_accuracy: 0.9388 - val_loss: 0.6559 - val_categorical_crossentropy: 0.6559 - val_categorical_accuracy: 0.826

KeyboardInterrupt: 

In [23]:
K.set_value(model.optimizer.lr, 0.000125)

hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=10, verbose=1,
    validation_data = valid_datagen,
    validation_steps = 34000 / batchsize,
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/10
10000/10000 [==============================] - 5909s 591ms/step - loss: 0.6478 - categorical_crossentropy: 0.6478 - categorical_accuracy: 0.8299 - top_3_accuracy: 0.9401 - val_loss: 0.6330 - val_categorical_crossentropy: 0.6330 - val_categorical_accuracy: 0.8356 - val_top_3_accuracy: 0.9418

Epoch 00001: saving model to xception/xception.01-0.6330.h5
Epoch 2/10
10000/10000 [==============================] - 5863s 586ms/step - loss: 0.6460 - categorical_crossentropy: 0.6460 - categorical_accuracy: 0.8300 - top_3_accuracy: 0.9406 - val_loss: 0.6437 - val_categorical_crossentropy: 0.6437 - val_categorical_accuracy: 0.8310 - val_top_3_accuracy: 0.9407

Epoch 00002: saving model to xception/xception.02-0.6437.h5
Epoch 3/10
10000/10000 [==============================] - 5850s 585ms/step - loss: 0.6458 - categorical_crossentropy: 0.6458 - categorical_accuracy: 0.8303 - top_3_accuracy: 0.9403 - val_loss: 0.6415 - val_categorical_crossentropy: 0.6415 - val_categorical_accuracy: 0.829

In [25]:
K.set_value(model.optimizer.lr, 0.0000675)

hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=5, verbose=1,
    validation_data = valid_datagen,
    validation_steps = 34000 / batchsize,
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/5
10000/10000 [==============================] - 5903s 590ms/step - loss: 0.6231 - categorical_crossentropy: 0.6231 - categorical_accuracy: 0.8363 - top_3_accuracy: 0.9430 - val_loss: 0.6395 - val_categorical_crossentropy: 0.6395 - val_categorical_accuracy: 0.8302 - val_top_3_accuracy: 0.9440

Epoch 00001: saving model to xception/xception.01-0.6395.h5
Epoch 2/5
10000/10000 [==============================] - 5921s 592ms/step - loss: 0.6123 - categorical_crossentropy: 0.6123 - categorical_accuracy: 0.8383 - top_3_accuracy: 0.9442 - val_loss: 0.6342 - val_categorical_crossentropy: 0.6342 - val_categorical_accuracy: 0.8345 - val_top_3_accuracy: 0.9420

Epoch 00002: saving model to xception/xception.02-0.6342.h5
Epoch 3/5
10000/10000 [==============================] - 5868s 587ms/step - loss: 0.6143 - categorical_crossentropy: 0.6143 - categorical_accuracy: 0.8383 - top_3_accuracy: 0.9439 - val_loss: 0.6296 - val_categorical_crossentropy: 0.6296 - val_categorical_accuracy: 0.8349 -

KeyboardInterrupt: 

In [46]:
for layer in model.layers[:80]:
   layer.trainable = False

model.compile(optimizer=Adam(lr=0.0002), loss='categorical_crossentropy',
            metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 55, 55, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 55, 55, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 55, 55, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [47]:
batchsize = 1024

hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=5, verbose=1,
    validation_data = valid_datagen,
    validation_steps = 34000 / batchsize,
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/5
10000/10000 [==============================] - 2605s 260ms/step - loss: 0.6146 - categorical_crossentropy: 0.6146 - categorical_accuracy: 0.8378 - top_3_accuracy: 0.9442 - val_loss: 0.6422 - val_categorical_crossentropy: 0.6422 - val_categorical_accuracy: 0.8311 - val_top_3_accuracy: 0.9444

Epoch 00001: saving model to xception/xception.01-0.6422.h5
Epoch 2/5
 1117/10000 [==>...........................] - ETA: 37:53 - loss: 0.6033 - categorical_crossentropy: 0.6033 - categorical_accuracy: 0.8401 - top_3_accuracy: 0.9454

KeyboardInterrupt: 

In [43]:
batchsize = 2048
K.set_value(model.optimizer.lr, 0.0001)

hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=1, verbose=1,
    validation_data = valid_datagen,
    validation_steps = 34000 / batchsize,
    callbacks = callbacks
)

Epoch 1/1
10000/10000 [==============================] - 1750s 175ms/step - loss: 0.6063 - categorical_crossentropy: 0.6063 - categorical_accuracy: 0.8400 - top_3_accuracy: 0.9446 - val_loss: 0.6804 - val_categorical_crossentropy: 0.6804 - val_categorical_accuracy: 0.8267 - val_top_3_accuracy: 0.9380

Epoch 00001: saving model to xception/xception.01-0.6804.h5


In [16]:
hist_df = pd.concat([pd.DataFrame(hist.history) for hist in hists], sort=True)
hist_df.index = np.arange(1, len(hist_df)+1)
fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(16, 10))
axs[0].plot(hist_df.val_categorical_accuracy, lw=5, label='Validation Accuracy')
axs[0].plot(hist_df.categorical_accuracy, lw=5, label='Training Accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].grid()
axs[0].legend(loc=0)
axs[1].plot(hist_df.val_categorical_crossentropy, lw=5, label='Validation MLogLoss')
axs[1].plot(hist_df.categorical_crossentropy, lw=5, label='Training MLogLoss')
axs[1].set_ylabel('MLogLoss')
axs[1].set_xlabel('Epoch')
axs[1].grid()
axs[1].legend(loc=0)
fig.savefig('hist.png', dpi=300)
plt.show();

ValueError: No objects to concatenate

In [41]:
model = load_model('xception/xception.07-0.6321.h5', custom_objects={'top_3_accuracy': top_3_accuracy, 'AdamAccumulate': AdamAccumulate})

In [43]:
def image_generator_valid(size, batchsize, ks, lw=6, time_color=True):
    while True:
        for k in np.random.permutation(ks):
            filename = os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(k))
            for df in pd.read_csv(filename, chunksize=batchsize):
                df['drawing'] = df['drawing'].apply(json.loads)
                x = np.zeros((len(df), size, size, 3))
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, :3] = draw_cv2(raw_strokes, size=size, lw=lw,
                                             time_color=time_color)
                x = preprocess_input(x).astype(np.float32)
                yield x
                
valid_predict = image_generator_xd_1(size=144, batchsize=512, ks=[NCSVS-1])

valid_predictions = model.predict_generator(valid_predict, steps=34000 / 256, verbose=1)
map3 = mapk(valid_df[['y']].values, preds2catids(valid_predictions).values)
print('Map3: {:.3f}'.format(map3))

133/132 [==============================] - 137s 1s/step
Map3: 0.876


In [44]:
print(valid_predictions.shape)
print(preds2catids(valid_predictions).values)

(68096, 340)
[[330  12 240]
 [ 39 338 255]
 [108  64  68]
 ...
 [322  71 320]
 [ 21 196 240]
 [ 53 254  94]]


## Create Submission

In [48]:
batchsize=512
test = pd.read_csv(os.path.join(INPUT_DIR, 'test_simplified.csv'))
test_datagen = image_generator_test(size=144, batchsize=batchsize)

In [49]:
test_predictions = model.predict_generator(test_datagen, steps = (112199 / batchsize), verbose=1)
print(test_predictions.shape)

220/219 [==============================] - 223s 1s/step
(112199, 340)


In [50]:
top3 = preds2catids(test_predictions)
top3.head()
top3.shape

cats = list_all_categories()
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(cats)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

,a,b,c
0,234,281,266
1,144,36,226
2,305,62,110
3,187,82,304
4,56,113,112


(112199, 3)

,a,b,c
0,radio,stereo,snorkel
1,hockey_puck,bottlecap,pool
2,The_Great_Wall_of_China,castle,fence
3,mountain,crayon,The_Eiffel_Tower
4,campfire,fireplace,fire_hydrant


(112199, 3)

In [51]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.to_csv('xception_submission_{}_final_blend_3.csv'.format(int(map3 * 10**4)), index=False)
submission.head()
submission.shape

,key_id,word
0,9000003627287624,radio stereo snorkel
1,9000010688666847,hockey_puck bottlecap pool
2,9000023642890129,The_Great_Wall_of_China castle fence
3,9000038588854897,mountain crayon The_Eiffel_Tower
4,9000052667981386,campfire fireplace fire_hydrant


(112199, 2)

In [19]:
end = dt.datetime.now()
print('Latest run {}.\nTotal time {}s'.format(end, (end - start).seconds))

Latest run 2018-12-04 00:37:22.330471.
Total time 267s
